# DrivenData
## Pump it up

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import copy
data = pd.read_csv('./DrivenData/training_values.csv', nrows=10000)
holdout = pd.read_csv('./DrivenData/test_values.csv')
y = pd.read_csv('./DrivenData/training_labels.csv', nrows=10000)
y = y['status_group']
#pd.value_counts(y).plot.bar()

In [10]:
data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [9]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(data,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(data.columns)

feature_scores = pd.concat([dfcolumns,dfscores], axis=1)
feature_scores.columns = ['Specs', 'Score']
print(feature_scores.nlargest(10,'Score'))

ValueError: could not convert string to float: '2011-03-14'

In [2]:
numerics = ['int16', 'int32','int64','float16','float32', 'float64']
NUMERIC_COLUMNS = data.select_dtypes(include=numerics)
ALPHANUMERIC_COLUMNS = data.select_dtypes(include=['object'])
#ALPHANUMERIC_COLUMNS.describe()
#ALPHANUMERIC_COLUMNS.head()
#NUMERIC_COLUMNS.describe()

In [3]:
train_objs_num = len(data)
dataset_new = pd.concat(objs=[data, holdout], axis=0)
dataset_new = dataset_new.drop(['id'], axis=1)
dataset_new.fillna("", inplace=True)
dataset_new = pd.get_dummies(dataset_new)
train = copy.copy(dataset_new[:train_objs_num])
test = copy.copy(dataset_new[train_objs_num:])

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=42)

In [5]:
from sklearn.svm import LinearSVC
svm = LinearSVC()
svm.fit(X_train, y_train)

c:\users\jespe\.virtualenvs\machinelearningnotebook-qxxtnzx7\lib\site-packages\sklearn\svm\_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [6]:
print("train score lin_svc:", svm.score(X_train,y_train))
print("test score lin_svc: ", svm.score(X_test,y_test))


train score lin_svc: 0.77125
test score lin_svc:  0.64875


In [7]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss='hinge', penalty='l2')
sgd.fit(X_train,y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [8]:
print("train score sgd: ", sgd.score(X_train,y_train) )
print("test score sgd: ", sgd.score(X_test,y_test) )

train score sgd:  0.541875
test score sgd:  0.52875
